## Imports

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Sequential
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from string import punctuation
import json

## Data Preprocessing

In [2]:
# Load data
with open("./sentiment.txt") as f:
    reviews = f.read()
    
reviews

'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  \t0\nNot sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  \t0\nAttempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  \t0\nVery little music or anything to speak of.  \t0\nThe best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  \t1\nThe rest of the movie lacks art, charm, meaning... If it\'s about emptiness, it works I guess because it\'s empty.  \t0\nWasted two hours.  \t0\nSaw the movie today and thought it was a good effort, good messages for kids.  \t1\nA bit predictable.  \t0\nLoved the casting of Jimmy Buffet as the science teacher.  \t1\nAnd those baby owls were adorable.  \t1\nThe movie showed a lot of Florida at it\'s best, made it look very appealing.  \t1\nThe S

In [3]:
data = pd.DataFrame([review.split('\t') for review in reviews.split('\n')], columns=['Review', 'Sentiment'])
data.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [4]:
# Shuffle the data
data = data.sample(frac=1)
data.head()

,Review,Sentiment
2997,Item Does Not Match Picture.,0
2560,"Muddy, low quality sound, and the casing aroun...",0
1848,"To my disbelief, each dish qualified as the wo...",0
600,Perabo has a nice energy level and is obviousl...,1
406,I liked it.,1


In [5]:
def split_words_reviews(data):
    text = list(data['Review'].values)
    clean_text = []
    for t in text:
        clean_text.append(t.translate(str.maketrans('', '', punctuation)).lower().rstrip())
    tokenized = [word_tokenize(x) for x in clean_text]
    all_text = []
    for tokens in tokenized:
        for t in tokens:
            all_text.append(t)
    return tokenized, set(all_text)

In [6]:
# Tokenize
reviews, vocab = split_words_reviews(data)

print(reviews[0])
print(vocab)

['item', 'does', 'not', 'match', 'picture']
{'all', 'pile', 'scrimm', 'girl', 'assaulted', 'semi', 'baxendale', 'theatre', 'dialog', 'completed', '400', 'apology', 'through', 'hardest', 'pineapple', 'screamy', 'pm', 'infatuated', 'forward', 'masterpieces', 'coal', 'requested', 'never', 'riot', 'box', 'r', 'feature', 'messaging', 'tonight', 'valley', 'hype', 'expression', 'wood', 'farce', 'seperate', 'wings', 'cbr', '2160', 'gere', 'computer', 'postproduction', 'overhaul', 'sinister', 'uptight', 'unbelievably', 'n', 'menace', 'bitches', 'cheesecurds', 'honestly', 'alone', 'prone', 'orders', 'errol', 'stated', 'showed', 'believed', 'stopped', 'gake', 'equipment', 'death', 'stars', 'military', 'hostess', 'spices', 'seems', 'keypad', 'lion', 'cook', 'whom', 'america', 'reading', 'glad', 'phonethe', 'imagination', 'release', 'years', 'garlic', 'has', 'boost', 'purcashed', 'flimsy', 'nicer', 'display', 'sequence', 'giant', 'wasting', 'gluten', 'caused', 'satisfying', 'competent', 'freshness'

In [7]:
def create_dictionaries(words):
    word_to_int_dict = {w:i+1 for i, w in enumerate(words)}
    word_to_int_dict[''] = 0  # Ensuring the empty string is included and mapped to 0
    int_to_word_dict = {i:w for w, i in word_to_int_dict.items()}
    return word_to_int_dict, int_to_word_dict

In [8]:
# Creating vocabulary
word_to_int_dict, int_to_word_dict = create_dictionaries(vocab)

print(int_to_word_dict)
print(word_to_int_dict)

{1: 'all', 2: 'pile', 3: 'scrimm', 4: 'girl', 5: 'assaulted', 6: 'semi', 7: 'baxendale', 8: 'theatre', 9: 'dialog', 10: 'completed', 11: '400', 12: 'apology', 13: 'through', 14: 'hardest', 15: 'pineapple', 16: 'screamy', 17: 'pm', 18: 'infatuated', 19: 'forward', 20: 'masterpieces', 21: 'coal', 22: 'requested', 23: 'never', 24: 'riot', 25: 'box', 26: 'r', 27: 'feature', 28: 'messaging', 29: 'tonight', 30: 'valley', 31: 'hype', 32: 'expression', 33: 'wood', 34: 'farce', 35: 'seperate', 36: 'wings', 37: 'cbr', 38: '2160', 39: 'gere', 40: 'computer', 41: 'postproduction', 42: 'overhaul', 43: 'sinister', 44: 'uptight', 45: 'unbelievably', 46: 'n', 47: 'menace', 48: 'bitches', 49: 'cheesecurds', 50: 'honestly', 51: 'alone', 52: 'prone', 53: 'orders', 54: 'errol', 55: 'stated', 56: 'showed', 57: 'believed', 58: 'stopped', 59: 'gake', 60: 'equipment', 61: 'death', 62: 'stars', 63: 'military', 64: 'hostess', 65: 'spices', 66: 'seems', 67: 'keypad', 68: 'lion', 69: 'cook', 70: 'whom', 71: 'amer

In [9]:
with open('word_to_int_dict.json', 'w') as fp:
    json.dump(word_to_int_dict, fp)

In [10]:
print(np.max([len(x) for x in reviews]))
print(np.mean([len(x) for x in reviews]))

70
11.783333333333333


In [11]:
def pad_text(tokenized_reviews, seq_length):
    reviews = []
    for review in tokenized_reviews:
        if len(review) >= seq_length:
            reviews.append(review[:seq_length])
        else:
            reviews.append(['']*(seq_length-len(review)) + review)
    return np.array(reviews)

In [12]:
# Pad and encode reviews
seq_length = 50
padded_sentences = pad_text(reviews, seq_length)
encoded_sentences = np.array([[word_to_int_dict[word] for word in review] for review in padded_sentences])
encoded_sentences[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0, 4551, 5244, 4776, 4212, 3543])

In [13]:
# Prepare labels
labels = np.array([int(x) for x in data['Sentiment'].values])

## Model Building

In [14]:
class SentimentBiLSTM(Model):
    def __init__(self, n_vocab, n_embed, n_hidden, n_output, n_layers, drop_p=0.8):
        super(SentimentBiLSTM, self).__init__()
        self.embedding = layers.Embedding(input_dim=n_vocab, output_dim=n_embed)
        self.lstm = layers.Bidirectional(
            layers.LSTM(n_hidden, return_sequences=True, dropout=drop_p)
        )
        self.dropout = layers.Dropout(drop_p)
        self.fc = layers.Dense(n_output, activation='sigmoid')

    def call(self, x):
        x = self.embedding(x)
        lstm_out = self.lstm(x)  # Only one value is returned
        x = self.dropout(lstm_out)
        x = self.fc(x)
        return x[:, -1]

n_vocab = len(word_to_int_dict)
n_embed = 50
n_hidden = 100
n_output = 1
n_layers = 2

# Instantiate the model
model = SentimentBiLSTM(n_vocab, n_embed, n_hidden, n_output, n_layers)

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

## Model Training

In [15]:
train_ratio = 0.8
valid_ratio = (1 - train_ratio) / 2

total = len(encoded_sentences)
train_cutoff = int(total * train_ratio)
valid_cutoff = int(total * (1 - valid_ratio))

train_x, train_y = encoded_sentences[:train_cutoff], labels[:train_cutoff]
valid_x, valid_y = encoded_sentences[train_cutoff:valid_cutoff], labels[train_cutoff:valid_cutoff]
test_x, test_y = encoded_sentences[valid_cutoff:], labels[valid_cutoff:]

batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(batch_size)
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(batch_size)

In [16]:
model.fit(train_dataset, validation_data=valid_dataset, epochs=15)

Epoch 1/15
75/75 [==============================] - 9s 60ms/step - loss: 0.6937 - accuracy: 0.5125 - val_loss: 0.6920 - val_accuracy: 0.5400
Epoch 2/15
75/75 [==============================] - 6s 86ms/step - loss: 0.6891 - accuracy: 0.5454 - val_loss: 0.6862 - val_accuracy: 0.6000
Epoch 3/15
75/75 [==============================] - 10s 134ms/step - loss: 0.6582 - accuracy: 0.6571 - val_loss: 0.6432 - val_accuracy: 0.6733
Epoch 4/15
75/75 [==============================] - 7s 94ms/step - loss: 0.5657 - accuracy: 0.7271 - val_loss: 0.5759 - val_accuracy: 0.7067
Epoch 5/15
75/75 [==============================] - 8s 108ms/step - loss: 0.4173 - accuracy: 0.8308 - val_loss: 0.4924 - val_accuracy: 0.7600
Epoch 6/15
75/75 [==============================] - 6s 86ms/step - loss: 0.3134 - accuracy: 0.8854 - val_loss: 0.5087 - val_accuracy: 0.7733
Epoch 7/15
75/75 [==============================] - 8s 102ms/step - loss: 0.2495 - accuracy: 0.9087 - val_loss: 0.4915 - val_accuracy: 0.7867
Epoch 8/1

In [17]:
model.summary()

Model: "sentiment_bi_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  270150    
                                                                 
 bidirectional (Bidirection  multiple                  120800    
 al)                                                             
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  201       
                                                                 
Total params: 391151 (1.49 MB)
Trainable params: 391151 (1.49 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Model Evaluation and Prediction

In [18]:
def preprocess_review(review, word_to_int_dict, seq_length=50):
    review = review.translate(str.maketrans('', '', punctuation)).lower().rstrip()
    tokenized = word_tokenize(review)
    if len(tokenized) >= seq_length:
        review = tokenized[:seq_length]
    else:
        review = ['']*(seq_length-len(tokenized)) + tokenized

    final = []
    for token in review:
        try:
            final.append(word_to_int_dict[token])
        except:
            final.append(word_to_int_dict[''])
    return np.array(final)

In [19]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.2f}")

10/10 [==============================] - 0s 30ms/step - loss: 0.3636 - accuracy: 0.8633
Test Loss: 0.3636
Test Accuracy: 0.86


In [20]:
def predict_review(review):
    review_encoded = preprocess_review(review, word_to_int_dict)
    review_encoded = np.expand_dims(review_encoded, axis=0)
    prediction = model.predict(review_encoded)
    msg = "This is a positive review." if prediction >= 0.5 else "This is a negative review."
    print(msg)
    print(f'Prediction = {prediction[0][0]}')

In [21]:
predict_review("Loved the casting of Jimmy Buffet as the science teacher.")
predict_review("There was absolutely no warmth or charm to these scenes or characters.")

1/1 [==============================] - 2s 2s/step
This is a positive review.
Prediction = 0.9998846054077148
1/1 [==============================] - 0s 35ms/step
This is a negative review.
Prediction = 0.009649661369621754


In [24]:
predict_review("I think the movie was up to the mark.")

1/1 [==============================] - 0s 42ms/step
This is a positive review.
Prediction = 0.7975245714187622
